In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import rdkit


In [3]:
tokenizer  = AutoTokenizer.from_pretrained("/auto/home/menuab/code/YNNtitan/torchtitan/tokenizers/Llama-3.2-chem-1B-v1")
model = AutoModelForCausalLM.from_pretrained("/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/77f5caabeda14ea6acbae86a/step-36000")

In [4]:
model.dtype, model.device

(torch.float32, device(type='cpu'))

In [28]:
canonical = rdkit.Chem.MolToSmiles(rdkit.Chem.MolFromSmiles("O=C1[N]c2ccncc2[CH][C@@H]1c1ccc(cc1)C"), canonical=True, isomericSmiles=False)
prompt = f"[SMILES]{canonical}[/SMILES]"
prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(model.device).input_ids
prompt

tensor([[128000, 128256,  98854,     16,  38154,   3100,     17,     58,   2198,
             60,     66,     18,  14606,  38154,     18,  23015,     60,     34,
             17,     28,     46,      8,    641,     16, 128257]])

In [29]:
output = model.generate(prompt, max_length=1000)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [30]:
tokenizer.batch_decode(output)

['<|begin_of_text|>[SMILES]Cc1ccc(C2[CH]c3cnccc3[N]C2=O)cc1[/SMILES][CONFORMER]C<0.8277, -0.0066, 0.0319>c<2.3413, -0.0208, 0.0049>1c<3.051, -1.2233, 0.1278>c<4.4475, -1.2513, 0.1028>c<5.1867, -0.074, -0.044>(C<6.7047, -0.076, -0.1051>2[C<7.3365, -0.9959, 0.8885>H]c<8.6831, -1.1645, 0.9338>3c<9.329, -2.0413, 1.905>n<10.5982, -2.2869, 1.9564>c<11.4053, -1.6519, 1.0095>c<10.9558, -0.7969, 0.0533>c<9.543, -0.5005, -0.0567>3[N<9.1351, 0.3242, -0.9924>]C<7.7969, 0.7051, -1.0522>2=O<7.4012, 1.516, -1.8781>)c<4.4824, 1.1365, -0.1692>c<3.0895, 1.1522, -0.1459>1[/CONFORMER][PROPERTY]3.13[/PROPERTY]<|end_of_text|>']